https://www.kaggle.com/code/manovirat/timeseries-using-prophet-hyperparameter-tuning

In [1]:
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.model_selection import ParameterGrid
from prophet.diagnostics import performance_metrics
import random
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

In [2]:
params_grid = {'seasonality_mode':('multiplicative','additive'),
               # set changepoint_prior_scale to 0.5 with steps of 0.005
               'changepoint_prior_scale':[x for x in np.arange(0.001, 0.5, 0.1)],
               'seasonality_prior_scale':[x for x in np.arange(0.01, 10, 2.5)],
              'holidays_prior_scale':[x for x in np.arange(0.01, 10, 2.5)]}

grid = ParameterGrid(params_grid)
cnt = 0
for p in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)

Total Possible Models 160


In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_30min_dataset.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date'],format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 


In [5]:
# Feature Engineering

# Add a column with the time
data['time'] = [x for x in range(0, len(data))]

# Rename column count to y
data = data.rename(columns={'count': 'y'})
data = data.rename(columns={'date': 'ds'})


train_data = data

In [6]:
cv_folds = TimeSeriesSplit(n_splits=5)

#convert data to only date not time
cv_data = train_data.copy()

#convert to timestamp with date format
#cv_data.ds = pd.to_datetime(cv_data.ds).dt.date

print(cv_data.ds)   

#Define cutoff dates for cross validation at end of train_data/cv_folds
cutoffs = []
# for train_index, test_index in cv_folds.split(train_data):
#     cutoffs.append(train_data.ds.iloc[test_index].max())
#     #convert to timestamp
#     cutoffs = pd.to_datetime(cutoffs)
    
# print(cutoffs)

for i, (train_index, test_index) in enumerate(cv_folds.split(cv_data)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    #get the date for the last train index
    cutoffs.append(cv_data.ds.iloc[train_index].max())
    #only date not time
    #cutoffs = pd.to_datetime(cutoffs).date

print(cutoffs)

# Define cutoffs for cross validation with 5 folds every two months with first cut on 2022-06-01, last day 2023-01-15 and horizon of 15 days
#cutoffs = pd.to_datetime(['2022-06-01', '2022-08-01', '2022-10-01', '2022-12-01', '2023-01-15'])
# print(cutoffs)


0        2022-04-01 00:00:00
1        2022-04-01 00:30:00
2        2022-04-01 01:00:00
3        2022-04-01 01:30:00
4        2022-04-01 02:00:00
                ...         
17515    2023-03-31 21:30:00
17516    2023-03-31 22:00:00
17517    2023-03-31 22:30:00
17518    2023-03-31 23:00:00
17519    2023-03-31 23:30:00
Name: ds, Length: 17520, dtype: object
Fold 0:
  Train: index=[   0    1    2 ... 2917 2918 2919]
  Test:  index=[2920 2921 2922 ... 5837 5838 5839]
Fold 1:
  Train: index=[   0    1    2 ... 5837 5838 5839]
  Test:  index=[5840 5841 5842 ... 8757 8758 8759]
Fold 2:
  Train: index=[   0    1    2 ... 8757 8758 8759]
  Test:  index=[ 8760  8761  8762 ... 11677 11678 11679]
Fold 3:
  Train: index=[    0     1     2 ... 11677 11678 11679]
  Test:  index=[11680 11681 11682 ... 14597 14598 14599]
Fold 4:
  Train: index=[    0     1     2 ... 14597 14598 14599]
  Test:  index=[14600 14601 14602 ... 17517 17518 17519]
['2022-05-31 19:30:00', '2022-07-31 15:30:00', '2022-09-30 11:

In [7]:
MASE_list = []
best_mase = 100000000
best_params = None
for p in grid:
    test = pd.DataFrame()
    print(p)
    random.seed(0)
    train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
                        holidays_prior_scale = p['holidays_prior_scale'],
                        seasonality_prior_scale = p['seasonality_prior_scale'],
                        seasonality_mode = p['seasonality_mode'],
                        weekly_seasonality=True,
                        daily_seasonality = True,
                        yearly_seasonality = False,
                        )
    
    train_model.add_regressor('time', standardize=False)


    train_model.fit(train_data)
    
    df_cv = cross_validation(train_model, period='60 days', horizon='30 days', parallel="processes")


    # X and y to pandas dataframe
    X_train = pd.DataFrame(df_cv)

    y_true = df_cv['y'].values
    #make y_true as dataframe
    y_true = pd.DataFrame(y_true)

    y_pred = df_cv['yhat'].values
    #make y_pred as dataframe
    y_pred = pd.DataFrame(y_pred)

    MASE_list.append(mean_absolute_scaled_error(y_true, y_pred))
    
    # print('RMSE:',np.sqrt(MSE))
    # RMSE_list.append(np.sqrt(MSE))

# Find the best parameters
tuning_results = pd.DataFrame(grid)
tuning_results['mase'] = MASE_list
print(tuning_results)

{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 0.01}


16:01:33 - cmdstanpy - INFO - Chain [1] start processing
16:01:33 - cmdstanpy - INFO - Chain [1] done processing
16:01:35 - cmdstanpy - INFO - Chain [1] start processing
16:01:35 - cmdstanpy - INFO - Chain [1] start processing
16:01:35 - cmdstanpy - INFO - Chain [1] start processing
16:01:35 - cmdstanpy - INFO - Chain [1] start processing
16:01:35 - cmdstanpy - INFO - Chain [1] start processing
16:01:35 - cmdstanpy - INFO - Chain [1] done processing
16:01:35 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
16:01:35 - cmdstanpy - INFO - Chain [1] done processing
16:01:35 - cmdstanpy - INFO - Chain [1] start processing
16:01:35 - cmdstanpy - INFO - Chain [1] done processing
16:01:36 - cmdstanpy - INFO - Chain [1] done processing
16:01:36 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
16:01:36 

{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 2.51}


16:02:35 - cmdstanpy - INFO - Chain [1] start processing
16:02:36 - cmdstanpy - INFO - Chain [1] done processing
16:02:36 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
16:02:36 - cmdstanpy - INFO - Chain [1] start processing
16:03:48 - cmdstanpy - INFO - Chain [1] done processing
16:03:49 - cmdstanpy - INFO - Chain [1] start processing
16:03:49 - cmdstanpy - INFO - Chain [1] start processing
16:03:49 - cmdstanpy - INFO - Chain [1] start processing
16:03:49 - cmdstanpy - INFO - Chain [1] start processing
16:03:49 - cmdstanpy - INFO - Chain [1] done processing
16:03:49 - cmdstanpy - INFO - Chain [1] start processing
16:03:49 - cmdstanpy - INFO - Chain [1] done processing
16:03:49 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
16:03:50 - cmdstanpy - INFO - Chain [1] done processing
16:03:50 

{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 5.01}


...

{'changepoint_prior_scale': 0.401, 'holidays_prior_scale': 7.51, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 5.01}


16:36:04 - cmdstanpy - INFO - Chain [1] start processing
16:36:04 - cmdstanpy - INFO - Chain [1] done processing
16:36:06 - cmdstanpy - INFO - Chain [1] start processing
16:36:06 - cmdstanpy - INFO - Chain [1] done processing
16:36:06 - cmdstanpy - INFO - Chain [1] start processing
16:36:06 - cmdstanpy - INFO - Chain [1] done processing
16:36:06 - cmdstanpy - INFO - Chain [1] start processing
16:36:06 - cmdstanpy - INFO - Chain [1] start processing
16:36:06 - cmdstanpy - INFO - Chain [1] done processing
16:36:07 - cmdstanpy - INFO - Chain [1] done processing
16:36:07 - cmdstanpy - INFO - Chain [1] start processing
16:36:07 - cmdstanpy - INFO - Chain [1] done processing


{'changepoint_prior_scale': 0.401, 'holidays_prior_scale': 7.51, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 7.51}


16:36:08 - cmdstanpy - INFO - Chain [1] start processing
16:36:08 - cmdstanpy - INFO - Chain [1] done processing
16:36:09 - cmdstanpy - INFO - Chain [1] start processing
16:36:10 - cmdstanpy - INFO - Chain [1] done processing
16:36:10 - cmdstanpy - INFO - Chain [1] start processing
16:36:10 - cmdstanpy - INFO - Chain [1] done processing
16:36:10 - cmdstanpy - INFO - Chain [1] start processing
16:36:10 - cmdstanpy - INFO - Chain [1] done processing
16:36:10 - cmdstanpy - INFO - Chain [1] start processing
16:36:11 - cmdstanpy - INFO - Chain [1] done processing
16:36:11 - cmdstanpy - INFO - Chain [1] start processing
16:36:11 - cmdstanpy - INFO - Chain [1] done processing


     changepoint_prior_scale  holidays_prior_scale seasonality_mode  \
0                      0.001                  0.01   multiplicative   
1                      0.001                  0.01   multiplicative   
2                      0.001                  0.01   multiplicative   
3                      0.001                  0.01   multiplicative   
4                      0.001                  0.01         additive   
..                       ...                   ...              ...   
155                    0.401                  7.51   multiplicative   
156                    0.401                  7.51         additive   
157                    0.401                  7.51         additive   
158                    0.401                  7.51         additive   
159                    0.401                  7.51         additive   

     seasonality_prior_scale      mase  
0                       0.01  1.265905  
1                       2.51  1.201250  
2                       

In [8]:
best_params = grid[np.argmin(MASE_list)]
print('Best Parameters',best_params)
print('Best MASE',np.min(MASE_list))
print(MASE_list)

Best Parameters {'seasonality_prior_scale': 2.51, 'seasonality_mode': 'multiplicative', 'holidays_prior_scale': 2.51, 'changepoint_prior_scale': 0.001}
Best MASE 0.9182860079436967
[1.265904740518625, 1.2012500673688014, 1.186236323214919, 1.1337631678585933, 1.1978254961720103, 1.1991393310031493, 1.1896061631740455, 1.258145798378226, 1.0391569023654537, 0.9182860079436967, 1.1874969582678494, 1.1916739737610407, 1.1933641685441656, 1.2032628291940368, 1.1976875278091959, 1.2102749726135935, 1.1173299131081167, 1.199493274176376, 0.9357839423421309, 1.201177567922289, 1.1979926889531163, 1.2120568657770796, 1.200155619572108, 1.1946476592599604, 1.0543872546184612, 1.196638161310955, 1.2019105987729526, 1.1808374782017257, 1.2082606461829932, 1.1871648663042176, 1.1958906910533122, 1.1992766143831697, 1.3291909168702507, 0.9217142893499395, 0.9220645503748028, 0.9492282784952644, 0.9385248779591674, 0.9392797350692957, 0.9394553231572417, 0.9394582152393842, 1.334955693880536, 0.9371